## 1) 라이브러리 임포트

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

## 2) 데이터 전처리

### 2.1) 웹 크롤링 후 DataFrame 생성 

In [2]:
def fetch_data():
    url = 'https://droneportal.or.kr/subList/22000000153'
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = urllib.request.Request(url, headers=headers)
    try:
        html_data = urllib.request.urlopen(req).read().decode('utf-8')
    except Exception as e:
        print("웹 페이지 요청 실패:", e)
        exit()
    soup = BeautifulSoup(html_data, 'html.parser')
    tables = soup.find_all('table')
    target_table = None
    for table in tables:
        if "UA" in table.get_text():
            target_table = table
            break
    if target_table is None:
        print("대상 테이블을 찾을 수 없습니다.")
        exit()
    rows = target_table.find_all("tr")
    data = []
    for row in rows:
        cells = row.find_all(["td", "th"])
        cell_texts = [cell.get_text(strip=True) for cell in cells][:4]
        while len(cell_texts) < 4:
            cell_texts.append(None)
        # 행의 어느 셀에라도 "UA"와 숫자가 있으면 포함
        if cell_texts and any(re.search(r'\bUA\s*\d+\b', cell) for cell in cell_texts if cell):
            data.append(cell_texts)
    columns = ["구역 코드", "위치", "수평범위", "수직범위"]
    return pd.DataFrame(data, columns=columns)

### 2.2.1) dms 형식 좌표 문자열을 십진수 좌표로 변환하는 함수

In [3]:
def dms_to_dd(dms_str):
    m = re.match(r"(\d{2,3})(\d{2})(\d{2})([NSEW])", dms_str.replace(" ", ""))
    if not m:
        return None
    deg, minute, second, direction = m.groups()
    dd = int(deg) + int(minute) / 60 + int(second) / 3600
    if direction in ["S", "W"]:
        dd *= -1
    return dd

### 2.2.2) 원형 좌표 데이터(Circle 데이터) 추출 함수

In [4]:
def extract_circle_data(text, region):
    m = re.search(
        r'Circle with radius of ([\d.]+)\s*(km|m).*?centered on\s*([\d]{6,7}[NSEW])\s*([\d]{6,7}[NSEW])',
        text
    )
    if m:
        radius = float(m.group(1))
        unit = m.group(2).lower()
        if unit == "m":
            radius /= 1000
        lat_str = m.group(3)
        lon_str = m.group(4)
        # 지역명이 "광주"인 경우에만 경도 교정 (예: 128... → 126...)
        if region and "광주" in region:
            if lon_str.startswith("128"):
                lon_str = "126" + lon_str[3:]
        return radius, dms_to_dd(lat_str), dms_to_dd(lon_str)
    return None, None, None

### 2.2.3) 원형 좌표 데이터(Circle 데이터) 설정 및 입력 함수

In [5]:
def build_circle_table():
    df = fetch_data()
    df['반경'] = None
    df['위도'] = None
    df['경도'] = None
    for idx, row in df.iterrows():
        text = row['수평범위']
        region = row['위치']  # 여기서는 '위치' 열에 지역명이 들어 있다고 가정
        if text and "Circle" in text:
            radius, lat, lon = extract_circle_data(text, region)
            df.at[idx, '반경'] = radius
            df.at[idx, '위도'] = lat
            df.at[idx, '경도'] = lon
    return df[df['반경'].notnull()][["구역 코드", "위치", "반경", "위도", "경도"]].reset_index(drop=True)

### 2.3.1) 다각형 좌표 데이터(NE 데이터) 추출 함수

In [6]:
def extract_ne_data(text):
    matches = re.findall(r'(\d{6,7}N)\s*(\d{6,7}E)', text)
    result = []
    for lat_str, lon_str in matches:
        lat = dms_to_dd(lat_str)
        lon = dms_to_dd(lon_str)
        result.append((lat, lon))
    return result if result else None

def order_coords(coords):
    if not coords or len(coords) <= 1:
        return coords
    cx = sum(pt[0] for pt in coords) / len(coords)
    cy = sum(pt[1] for pt in coords) / len(coords)
    return sorted(coords, key=lambda pt: math.atan2(pt[1]-cy, pt[0]-cx))

### 2.3.2) 다각형 좌표 데이터(NE 데이터) 설정 및 입력 함수

In [7]:
# NE Table 빌드 함수
def build_ne_table():
    df = fetch_data()
    df['NE'] = None
    for idx, row in df.iterrows():
        text = row['수평범위']
        if text and "Circle" not in text:
            ne_coords = extract_ne_data(text)
            if ne_coords and len(ne_coords) > 1:
                ne_coords = order_coords(ne_coords)
            df.at[idx, 'NE'] = ne_coords
    return df[df['NE'].notnull()][["구역 코드", "위치", "NE"]].reset_index(drop=True)

### 3.1) 지역 좌표 전처리 코드 실행 및 결과 출력

In [30]:
# 최종 지역 좌표 전처리 코드 실행
if __name__ == "__main__":
    circle_table = build_circle_table()
    ne_table = build_ne_table()
    print("=== Circle Table ===")
    print(circle_table)
    print("=== NE Table ===")
    print(ne_table)

=== Circle Table ===
      구역 코드   위치    반경         위도          경도
0      UA 2  구성산   1.8  35.739167    127.0075
1      UA 3   약산   0.7  35.739167  128.417222
2      UA 4  봉화산   4.0  35.625278  129.092222
3      UA 5  덕두산   4.5  35.411389    127.5325
4      UA 6   금산   2.1  34.736389  127.981111
5      UA 7   홍산   1.2  35.828056  127.081111
6     UA 10   고창   4.0  35.386389  126.731389
7     UA 21  방장산   3.0  35.449444  126.738056
8     UA 24   구좌   2.8  33.478056  126.822778
9     UA 27  미악산   1.2       33.3  126.554444
10    UA 28  서운산   2.0  36.930556  127.283056
11    UA 29   오천   2.0  36.953056  127.287778
12    UA 30   북좌   2.0     37.045  127.327778
13  * UA 32   퇴촌   0.3  37.466667    127.3025
14  * UA 40   고령  0.08  35.842778  128.444167
15  **UA 42   광주  0.05  35.221667  126.861667
=== NE Table ===
      구역 코드   위치                                                 NE
0      UA 9   양평  [(37.45, 127.38333333333334), (37.502777777777...
1     UA 14   공주  [(36.45805555555556, 126.9

### 3.2) csv 파일로 출력

In [ ]:
circle_table.to_csv('Circle_table.csv', index=False, encoding='cp949')
ne_table.to_csv('NE_table.csv', index=False, encoding='cp949')

## 4) 드론 공원 좌표 출력

### 4.1) 라이브러리 임포트

In [32]:
import os
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

### 4.2) 좌표 변환 함수

In [33]:
def parse_dms_extended(coord_str):
    """
    '°' 기호가 포함된 좌표 문자열(예: "37°32'47.9\"N") 또는 
    단순 DMS 형식(예: "362754N")을 소수점 좌표(십진수)로 변환합니다.
    """
    coord_str = coord_str.strip()
    if "°" in coord_str:
        pattern = r"(\d+)[°]\s*(\d+)[\'’]\s*(\d+(?:\.\d+)?)[\"']?\s*([NSEW])"
        match = re.search(pattern, coord_str)
        if match:
            deg, minute, second, direction = match.groups()
            dd = float(deg) + float(minute) / 60 + float(second) / 3600
            if direction in ['S', 'W']:
                dd = -dd
            return dd
        else:
            return None
    else:
        pattern = r"(\d{2,3})(\d{2})(\d{2})([NSEW])"
        m = re.match(pattern, coord_str.replace(" ", ""))
        if m:
            deg, minute, second, direction = m.groups()
            dd = int(deg) + int(minute) / 60 + int(second) / 3600
            if direction in ['S', 'W']:
                dd = -dd
            return dd
        return None

def extract_coordinates(text):
    """
    주어진 텍스트에서 좌표 문자열을 모두 추출합니다.
    - 먼저 ° 기호 포함 형식(예: "37°32'47.9\"N")을 우선적으로 찾고,
    - 없으면 숫자만 있는 DMS 형식(예: "362754N")을 찾습니다.
    """
    text = text.replace("–", " ")
    pattern2 = r"(\d+[°]\s*\d+[\'’]\s*\d+(?:\.\d+)?[\"']?\s*[NSEW])"
    matches2 = re.findall(pattern2, text)
    if matches2:
        return matches2
    else:
        pattern1 = r"(\d{6,7}[NSEW])"
        return re.findall(pattern1, text)

### 4.3) 웹페이지 크롤링

In [34]:
url = "https://droneportal.or.kr/subList/22000000157"
headers = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=headers)
with urllib.request.urlopen(req) as response:
    html_data = response.read().decode('utf-8')
soup = BeautifulSoup(html_data, 'html.parser')
page_text = soup.get_text(separator="\n")

### 4.4) 원하는 항목만 남기기

In [35]:
# 2. 대상 항목 (고양 대덕 드론 비행장 제외한 3개)만 남기기
target_titles = [
    "대전 금강변 드론공원",
    "광주 북구 영산강변 드론공원",
    "광나루 한강변 드론공원"
]

results = []
for i, title in enumerate(target_titles):
    start_idx = page_text.find(title)
    if start_idx == -1:
        continue
    if i < len(target_titles) - 1:
        next_idx = page_text.find(target_titles[i+1], start_idx + len(title))
        block = page_text[start_idx: next_idx] if next_idx != -1 else page_text[start_idx:]
    else:
        block = page_text[start_idx:]
    
    # 전체 블록에서 좌표 문자열 추출 (위치 텍스트가 따로 없을 수 있으므로)
    coord_strs = extract_coordinates(block)
    converted = [parse_dms_extended(cs) for cs in coord_strs if parse_dms_extended(cs) is not None]
    pairs = []
    for j in range(0, len(converted), 2):
        if j+1 < len(converted):
            pairs.append((converted[j], converted[j+1]))
    
    results.append({
        "지역": title,
        "위치": block.strip(),  # 원본 블록 텍스트를 위치로 사용 (추후 CSV의 위치 컬럼에 '지역' 값을 넣을 예정)
        "좌표": pairs
    })

df_new = pd.DataFrame(results, columns=["지역", "위치", "좌표"])
print("추출된 신규 데이터:")
print(df_new)

추출된 신규 데이터:
                지역                                                 위치  \
0      대전 금강변 드론공원  대전 금강변 드론공원\n\n\n\n\n\n\n\n\n\n\n\n\n대전 금강변 드론...   
1  광주 북구 영산강변 드론공원  광주 북구 영산강변 드론공원\n\n\n\n\n\n\n\n\n\n\n\n\n광주 북구...   
2     광나루 한강변 드론공원  광나루 한강변 드론공원\n\n\n\n\n\n\n\n\n\n\n\n\n광나루 한강변 ...   

                                                  좌표  
0  [(36.465, 127.39055555555557), (36.46583333333...  
1          [(35.22166666666667, 126.86166666666666)]  
2  [(37.546638888888886, 127.12052777777777), (12...  


### 4.5) 기존의 csv 파일에 데이터 추가 - NE table

In [36]:
# 3. CSV 파일에 추가
# (1) NE_table.csv: 대전 금강변 드론공원의 데이터 → [위치, NE]
df_NE_new = df_new[df_new["지역"] == "대전 금강변 드론공원"].copy()
# '위치' 컬럼은 신규 데이터의 '지역' 값로 설정
df_NE_new["위치"] = df_NE_new["지역"]
# 'NE' 컬럼은 좌표 쌍을 문자열로 저장 (전체 쌍 목록)
df_NE_new["NE"] = df_NE_new["좌표"].apply(lambda x: str(x))
df_NE_new = df_NE_new[["위치", "NE"]]

if os.path.exists("NE_table.csv"):
    try:
        df_NE_old = pd.read_csv("NE_table.csv", encoding="utf-8-sig")
    except UnicodeDecodeError:
        df_NE_old = pd.read_csv("NE_table.csv", encoding="cp949")
else:
    df_NE_old = pd.DataFrame(columns=["위치", "NE"])
df_NE = pd.concat([df_NE_old, df_NE_new], ignore_index=True)
try:
    df_NE.to_csv("NE_table.csv", index=False, encoding="utf-8-sig")
except PermissionError:
    print("NE_table.csv 파일에 쓰기 권한이 없습니다. 파일을 닫은 후 다시 실행하세요.")
    exit()

### 4.6) 기존의 csv 파일에 데이터 추가 - Circle table

In [37]:
df_Circle_new = df_new[df_new["지역"].isin(["광주 북구 영산강변 드론공원", "광나루 한강변 드론공원"])].copy()
# '위치' 컬럼은 신규 데이터의 '지역' 값로 설정
df_Circle_new["위치"] = df_Circle_new["지역"]
def get_first_pair(pair_list):
    if isinstance(pair_list, list) and len(pair_list) > 0:
        return pair_list[0]
    return (None, None)
df_Circle_new["위도"] = df_Circle_new["좌표"].apply(lambda x: get_first_pair(x)[0])
df_Circle_new["경도"] = df_Circle_new["좌표"].apply(lambda x: get_first_pair(x)[1])
df_Circle_new = df_Circle_new[["위치", "위도", "경도"]]

if os.path.exists("Circle_table.csv"):
    try:
        df_Circle_old = pd.read_csv("Circle_table.csv", encoding="utf-8-sig")
    except UnicodeDecodeError:
        df_Circle_old = pd.read_csv("Circle_table.csv", encoding="cp949")
else:
    df_Circle_old = pd.DataFrame(columns=["위치", "위도", "경도"])
df_Circle = pd.concat([df_Circle_old, df_Circle_new], ignore_index=True)
try:
    df_Circle.to_csv("Circle_table.csv", index=False, encoding="utf-8-sig")
except PermissionError:
    print("Circle_table.csv 파일에 쓰기 권한이 없습니다. 파일을 닫은 후 다시 실행하세요.")
    exit()

In [ ]:
import os
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

def parse_dms_extended(coord_str):
    """
    '°' 기호가 포함된 좌표 문자열(예: "37°32'47.9\"N") 또는 
    단순 DMS 형식(예: "362754N")을 소수점 좌표(십진수)로 변환합니다.
    """
    coord_str = coord_str.strip()
    if "°" in coord_str:
        pattern = r"(\d+)[°]\s*(\d+)[\'’]\s*(\d+(?:\.\d+)?)[\"']?\s*([NSEW])"
        match = re.search(pattern, coord_str)
        if match:
            deg, minute, second, direction = match.groups()
            dd = float(deg) + float(minute) / 60 + float(second) / 3600
            if direction in ['S', 'W']:
                dd = -dd
            return dd
        else:
            return None
    else:
        pattern = r"(\d{2,3})(\d{2})(\d{2})([NSEW])"
        m = re.match(pattern, coord_str.replace(" ", ""))
        if m:
            deg, minute, second, direction = m.groups()
            dd = int(deg) + int(minute) / 60 + int(second) / 3600
            if direction in ['S', 'W']:
                dd = -dd
            return dd
        return None

def extract_coordinates(text):
    """
    주어진 텍스트에서 좌표 문자열을 모두 추출합니다.
    - 먼저 ° 기호 포함 형식(예: "37°32'47.9\"N")을 우선적으로 찾고,
    - 없으면 숫자만 있는 DMS 형식(예: "362754N")을 찾습니다.
    """
    text = text.replace("–", " ")
    pattern2 = r"(\d+[°]\s*\d+[\'’]\s*\d+(?:\.\d+)?[\"']?\s*[NSEW])"
    matches2 = re.findall(pattern2, text)
    if matches2:
        return matches2
    else:
        pattern1 = r"(\d{6,7}[NSEW])"
        return re.findall(pattern1, text)

# 1. 페이지 크롤링 및 전체 텍스트 추출
url = "https://droneportal.or.kr/subList/22000000157"
headers = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=headers)
with urllib.request.urlopen(req) as response:
    html_data = response.read().decode('utf-8')
soup = BeautifulSoup(html_data, 'html.parser')
page_text = soup.get_text(separator="\n")

# 2. 대상 항목 (고양 대덕 드론 비행장 제외한 3개)만 남기기
target_titles = [
    "대전 금강변 드론공원",
    "광주 북구 영산강변 드론공원",
    "광나루 한강변 드론공원"
]

results = []
for i, title in enumerate(target_titles):
    start_idx = page_text.find(title)
    if start_idx == -1:
        continue
    if i < len(target_titles) - 1:
        next_idx = page_text.find(target_titles[i+1], start_idx + len(title))
        block = page_text[start_idx: next_idx] if next_idx != -1 else page_text[start_idx:]
    else:
        block = page_text[start_idx:]
    
    # 전체 블록에서 좌표 문자열 추출 (위치 텍스트가 따로 없을 수 있으므로)
    coord_strs = extract_coordinates(block)
    converted = [parse_dms_extended(cs) for cs in coord_strs if parse_dms_extended(cs) is not None]
    pairs = []
    for j in range(0, len(converted), 2):
        if j+1 < len(converted):
            pairs.append((converted[j], converted[j+1]))
    
    results.append({
        "지역": title,
        "위치": block.strip(),  # 원본 블록 텍스트를 위치로 사용 (추후 CSV의 위치 컬럼에 '지역' 값을 넣을 예정)
        "좌표": pairs
    })

df_new = pd.DataFrame(results, columns=["지역", "위치", "좌표"])
print("추출된 신규 데이터:")
print(df_new)

## 4) 지역명 통일화

### 4.1.1) Circle 데이터(원형 좌표) 불러오기

In [9]:
df_circle = build_circle_table()

### 4.1.2) 추후 지역명 match를 위해 지역명 수정

In [10]:
# 위치 변경 매핑을 정의
location_map = {
    '구성산': '전주(구성산)',
    '약산': '대구(약산)',
    '봉화산': '울산(봉화산)',
    '덕두산': '남원(덕두산)',
    '홍산': '전주(홍산)',
    '방장산': '광주(방장산)',
    '구좌': '제주(구좌)',
    '미악산': '제주(지악산)',
    '서운산': '천안(서운산)',
    '오천': '천안(오천)',
    '북좌': '천안(북좌)',
    '퇴촌': '양평(퇴촌)',
    '고령': '대구(고령)',
    '광주 북구 영산강변 드론공원': '광주(광주 북구 영산강변 드론공원)',
    '광나루 한강변 드론공원': '서울(광나루 한강변 드론공원)'
}

# '위치' 컬럼에 있는 값들을 매핑된 값으로 변경
df_circle['위치'] = df_circle['위치'].map(location_map).fillna(df_circle['위치'])

### 4.1.3) 지역명 수정 값 확인 - Circle 데이터

In [11]:
print(df_circle.head())

  구역 코드       위치   반경         위도          경도
0  UA 2  전주(구성산)  1.8  35.739167    127.0075
1  UA 3   대구(약산)  0.7  35.739167  128.417222
2  UA 4  울산(봉화산)  4.0  35.625278  129.092222
3  UA 5  남원(덕두산)  4.5  35.411389    127.5325
4  UA 6       금산  2.1  34.736389  127.981111


### 4.2.1) NE 데이터(다각형 좌표) 불러오기

In [12]:
df_NE = build_ne_table()

### 4.2.2) 추후 지역명 match를 위해 지역명 수정

In [13]:
# 위치 변경 매핑을 정의
location_map = {
    '공주': '세종(공주)',
    '시화호': '수원(시화호)',
    '하동': '광양(하동)',
    '장암산': '영월(장암산)',
    '청라': '인천(청라)',
    '병천천': '청주(병천천)',
    '미호천': '청주(미호천)',
    '울주': '울산(울주)',
    '김제': '군산(김제)',
    '대전 금강변 드론공원': '대전(대전 금강변 드론공원)'
}

# '위치' 컬럼에 있는 값들을 매핑된 값으로 변경
df_NE['위치'] = df_NE['위치'].map(location_map).fillna(df_NE['위치'])

### 4.3.3) 지역명 수정 값 확인 - NE 데이터

In [14]:
print(df_NE.head())

   구역 코드       위치                                                 NE
0   UA 9       양평  [(37.45, 127.38333333333334), (37.502777777777...
1  UA 14   세종(공주)  [(36.45805555555556, 126.96388888888889), (36....
2  UA 19  수원(시화호)  [(37.212500000000006, 126.67472222222223), (37...
3  UA 25   광양(하동)  [(34.98777777777778, 127.72333333333333), (35....
4  UA 26  영월(장암산)  [(37.36972222222222, 128.39194444444445), (37....


## 5) 기상 데이터 전처리

### 5.1.1) 기상 데이터 지역별 전처리를 위한 파일 불러오기

In [15]:
# 1. CSV 파일 읽어오기 (다른 인코딩 시도)
file_path = "C:/Users/user/Desktop/python 정리집/웹크롤링 팀프로젝트/기상데이터/weather_yn.csv"

# euc-kr 또는 utf-8-sig 시도 (한글 깨짐 해결)
df_weather = pd.read_csv(file_path, encoding='euc-kr') 

### 5.1.2) 데이터 구조 확인

In [16]:
print(df_weather)

      stnId stnNm   yyyymm  days_in_month  yn_month  avg_yn_month
0        90    속초  2024-01             31        17      0.548387
1        90    속초  2024-02             29        13      0.448276
2        90    속초  2024-03             31        18      0.580645
3        90    속초  2024-04             30        25      0.833333
4        90    속초  2024-05             31        24      0.774194
...     ...   ...      ...            ...       ...           ...
1147    295    남해  2024-08             31        28      0.903226
1148    295    남해  2024-09             30        24      0.800000
1149    295    남해  2024-10             31        20      0.645161
1150    295    남해  2024-11             30        25      0.833333
1151    295    남해  2024-12             31        31      1.000000

[1152 rows x 6 columns]


### 5.1.3) 지역별 월별 종합 수치를 그룹화

In [17]:
pivot_table = df_weather.pivot_table(index='stnNm', columns='yyyymm', values='yn_month', aggfunc='sum', fill_value=0)

### 5.1.4) 지역별 종합 컬럼 추가

In [18]:
pivot_table['합계'] = pivot_table.sum(axis=1)

### 5.1.5) 데이터 결과 확인

In [19]:
print(pivot_table)

yyyymm  2024-01  2024-02  2024-03  2024-04  2024-05  2024-06  2024-07  \
stnNm                                                                   
강릉           20       13       21       24       26       21       14   
강진군          15       14       18       20       22       23       13   
강화           10       12       24       24       24       23       11   
거제           20       15       21       21       22       21       10   
거창           13       15       18       20       20       21       15   
...         ...      ...      ...      ...      ...      ...      ...   
합천           17       16       22       21       21       23       12   
해남           17       13       18       19       22       23       13   
홍성            9       16       20       22       23       25       12   
홍천            8       12       20       24       24       21       12   
흑산도          16       13       20       22       25       24       12   

yyyymm  2024-08  2024-09  2024-10  2024-11  2024-1

### 5.2.1) 기상 데이터 및 지역 데이터 불러오기

In [20]:
circle_df = build_circle_table()
ne_df = build_ne_table()
total_df = df_weather

### 5.2.2) Circle 데이터와 NE 데이터에서 지역명 추출

In [21]:
circle_df['위치_앞두자리'] = circle_df['위치'].str[:2]
ne_df['위치_앞두자리'] = ne_df['위치'].str[:2]

### 5.2.3) 기상 데이터에서 지역명 추출

In [22]:
total_df['stnNm_앞두자리'] = total_df['stnNm'].str[:2]

### 5.2.4) Circle 데이터와 NE 데이터에서 일치하는 지역명 필터링

In [23]:
circle_filtered = circle_df[circle_df['위치_앞두자리'].isin(total_df['stnNm_앞두자리'])]
ne_filtered = ne_df[ne_df['위치_앞두자리'].isin(total_df['stnNm_앞두자리'])]

### 5.2.5) 필터링한 지역명 합쳐서 다시 필터링

In [24]:
merged_filtered = pd.concat([circle_filtered, ne_filtered], ignore_index=True)
final_filtered = total_df[total_df['stnNm_앞두자리'].isin(merged_filtered['위치_앞두자리'])]

### 5.2.6) 테이블 정제 작업

In [25]:
final_filtered = final_filtered.reset_index(drop=True)
final_filtered = final_filtered.drop(columns=['stnNm_앞두자리'])
print(final_filtered)

     stnId stnNm   yyyymm  days_in_month  yn_month  avg_yn_month
0      121    영월  2024-01             31        12      0.387097
1      121    영월  2024-02             29        15      0.517241
2      121    영월  2024-03             31        18      0.580645
3      121    영월  2024-04             30        25      0.833333
4      121    영월  2024-05             31        21      0.677419
..     ...   ...      ...            ...       ...           ...
127    288    밀양  2024-08             31        21      0.677419
128    288    밀양  2024-09             30        25      0.833333
129    288    밀양  2024-10             31        20      0.645161
130    288    밀양  2024-11             30        26      0.866667
131    288    밀양  2024-12             31        30      0.967742

[132 rows x 6 columns]


## 6) 최종 데이터 csv 파일

In [26]:
# circle_table.to_csv
# ne_table.to_csv